In [ ]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from lib import app
from lib.utils import load_points, save_points, load_json, save_json, load_dlc_points_as_df, load_scene, load_camera
from lib.plotting import plot_extrinsics
from lib.calib import triangulate_points, project_points

plt.style.use(os.path.join("..", "configs", "mplstyle.yaml"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
def convert_points(json_paths: list, out_fname: str, pts_per_frame = 5, frame_offset = 0, calib_points = True):
    data = load_json(json_paths[0])
    output = {
        "board_shape": [pts_per_frame, 1],
        "board_square_len": 0,
        "camera_resolution": [data["imageWidth"], data["imageHeight"]],
        "points": {}
    }
    for fname in json_paths:
        data = load_json(fname)
        if calib_points:
            assert len(data["shapes"]) == 9, f"File {fname} did not produce enough points"
        frame_num = int((data["imagePath"].split(".png")[0]).split("frame")[1])
        frame_num += frame_offset
        output["points"][f"frame{frame_num}.png"] = [[np.nan, np.nan]] * pts_per_frame
        for i, point in enumerate(data["shapes"]):
            if calib_points:
                assert f"p{i+1}" == point["label"]
            p_idx = int(point["label"][-1]) - 1
            assert 0 <= p_idx < pts_per_frame, f"Incorrect point number used: ({p_idx}, {pts_per_frame})"
            output["points"][f"frame{frame_num}.png"][p_idx] = (point["points"][0])

    save_json(out_fname, output)

In [ ]:
# Convert labelme points to acinoset points.
# Convert intrinsic points.
int_path = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/manual_labels"
cam1_path = sorted(glob(os.path.join(int_path, "599", '*.json')))
cam2_path = sorted(glob(os.path.join(int_path, "598", '*.json')))

convert_points(cam1_path, os.path.join(os.path.dirname(int_path), "cam1", "points.json"), pts_per_frame=9, frame_offset=0)
convert_points(cam2_path, os.path.join(os.path.dirname(int_path), "cam2", "points.json"), pts_per_frame=9, frame_offset=0)

In [ ]:
# Gather DLC labels of tail and spine and obtain pairwise points between the cameras to obtain an estimate of the extrinsics.
def get_pairwise_points_from_df(points_2d_df, camera_pairs):
    df_list = []
    # get pairwise estimates
    for cam_a, cam_b in camera_pairs:
        d0 = points_2d_df[points_2d_df['camera'] == cam_a]
        d1 = points_2d_df[points_2d_df['camera'] == cam_b]
        intersection_df = d0.merge(d1, how='inner', on=['frame', 'marker'], suffixes=('_a', '_b'))
        if intersection_df.shape[0] > 0:
            print(f'Found {intersection_df.shape[0]} pairwise points between camera {cam_a} and {cam_b}')
            df_list.append(intersection_df)
        else:
            print(f'No pairwise points between camera {cam_a} and {cam_b}')

    return df_list


def get_points_file(n_pts, fname):
    try:
        output = load_json(fname)
    except FileNotFoundError:
        output = {
            "board_shape": [n_pts, 1],
            "board_square_len": 0,
            "camera_resolution": [0, 0],
            "points": {}
        }

    return output


def generate_pairwise_points(markers, points_df, cam_a_idx, cam_b_idx, out_dir):
    output_a = get_points_file(len(markers), os.path.join(out_dir, f"points{cam_a_idx+1}.json"))
    output_b = get_points_file(len(markers), os.path.join(out_dir, f"points{cam_b_idx+1}.json"))
    frames = points_df["frame"].unique()
    for fno in frames:
        temp_a = []
        temp_b = []
        for m in markers:
            pts_a = points_df.query(f"frame == {fno} and marker=='{m}'")[["x_a", "y_a"]].values
            pts_b = points_df.query(f"frame == {fno} and marker=='{m}'")[["x_b", "y_b"]].values
            temp_a.append(pts_a.squeeze().tolist() if len(pts_a) > 0 else [np.nan, np.nan])
            temp_b.append(pts_b.squeeze().tolist() if len(pts_b) > 0 else [np.nan, np.nan])
        if len(np.asarray(temp_a, object).shape) == 2 and len(np.asarray(temp_b, object).shape) == 2:
            # np.any(~np.isnan(temp_a)) and np.any(~np.isnan(temp_b))
            output_a["points"][f"frame{fno}.png"] = temp_a
            output_b["points"][f"frame{fno}.png"] = temp_b

    save_json(os.path.join(out_dir, f"points{cam_a_idx+1}.json"), output_a)
    save_json(os.path.join(out_dir, f"points{cam_b_idx+1}.json"), output_b)

def plot_points_2d(points_fpath):

    points, _, _, _, cam_res = load_points(points_fpath)
    plt.figure()
    for i in range(points.shape[1]):
        plt.scatter(points[:, i, 0], points[:, i, 1], label=f"p{i+1}")
    plt.xlim((0, cam_res[0]))
    plt.ylim((0, cam_res[1]))
    plt.legend()
    plt.gca().invert_yaxis()
    plt.show(block=False)


def plot_points_3d(obj_pts, title):
    fig = plt.figure()
    gs = fig.add_gridspec(2, 3)
    # fig, ax = plt.subplots(2, 3)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2])
    # ax[0, 0].plot(obj_pts[:, 0])
    ax1.plot(obj_pts[:, 0])
    ax1.set_title("X")
    # ax[0, 1].plot(obj_pts[:, 1])
    ax2.plot(obj_pts[:, 1])
    ax2.set_title("Y")
    # ax[0, 2].plot(obj_pts[:, 2])
    ax3.plot(obj_pts[:, 1])
    ax3.set_title("Z")

    # ax = fig.add_subplot(2, 3, (4, 5, 6), projection='3d')
    ax4 = fig.add_subplot(gs[1, :], projection='3d')
    ax4.scatter(obj_pts[:, 0], obj_pts[:, 1], obj_pts[:, 2])
    ax4.set_xlabel("X")
    ax4.set_ylabel("Y")
    ax4.set_zlabel("Z")

    plt.title(title)
    plt.show(block=False)


In [ ]:
# Load DLC points.
dlc_dir = "/Users/zico/Library/CloudStorage/OneDrive-UniversityofCapeTown/msc/data/cheetah_videos/kinetic_dataset/2009_09_07/arabia/trial06/dlc_hand_labeled"
points_dir = "/Users/zico/Downloads"
n_cams = 4
dlc_points_fpaths = sorted(glob(os.path.join(dlc_dir, '*.h5')))
points_2d_df = load_dlc_points_as_df(dlc_points_fpaths, hand_labeled=True, verbose=False)
# filtered_points_2d_df = points_2d_df[points_2d_df['likelihood'] > 0.8]  # ignore points with low likelihood
filtered_points_2d_df = points_2d_df
filtered_points_2d_df = filtered_points_2d_df[['camera', 'frame', 'marker', 'x', 'y']]
# Add synchronisation offset to the third camera to sync it with cam1 and cam2.
filtered_points_2d_df = filtered_points_2d_df[filtered_points_2d_df["frame"] > 50]
filtered_points_2d_df.loc[filtered_points_2d_df['camera'] == 2, "frame"] -= 5
filtered_points_2d_df.loc[filtered_points_2d_df['camera'] == 3, "frame"] -= 48
camera_pairs = [[i % n_cams, (i + 1) % n_cams] for i in range(n_cams)]
df_list = get_pairwise_points_from_df(filtered_points_2d_df, camera_pairs)
markers = ["nose"]
for i, (cam_a, cam_b) in enumerate(camera_pairs):
    points_df = df_list[i]
    generate_pairwise_points(markers, points_df, cam_a, cam_b, points_dir)

In [ ]:
# Plot points to visually inspect the 2D points used in the optimisation.
plot_points_2d(
    os.path.join(
        "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/2009_09_11/extrinsic_calib/stereo/points",
        "points2.json"))


In [ ]:
# Intrinsic calibration.
cam_path = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/cam2"
K, D, R, t, used_points = app.calibrate_arabia_intrinsics(os.path.join(cam_path, "points.json"),
                                                          os.path.join(cam_path, "camera.json"),
                                                          f=(1042.0, 1042.0),
                                                          c=(640.0, 280.0),
                                                          reduce_points=False)

In [ ]:
# Determine the orientation of the force plates with respect to the cameras.
# Then adjust the camera pose so that the first force plate is the world frame.
data = load_json(
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/manual_labels/599/frame1.json"
)
img_pts = []
for i, point in enumerate(data["shapes"]):
    img_pts.append(point["points"][0])
k, d, _ = load_camera(
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/cam1/camera.json"
)
img_pts = np.array(img_pts)
obj_pts = np.zeros((6, 3), np.float32)
obj_pts[0, :2] = [2.1, -0.45]
obj_pts[1, :2] = [2.1, 0.45]
obj_pts[2, :2] = [2.7, 0.45]
obj_pts[3, :2] = [2.7, -0.45]
obj_pts[4, :2] = [1.5, 0.45]
obj_pts[5, :2] = [1.5, -0.45]
# obj_pts[6, :2] = [0.9, -0.45]
# obj_pts[7, :2] = [0.9, 0.45]
# obj_pts[8, :2] = [0.3, -0.45]
# obj_pts[9, :2] = [-0.3, 0.45]
init_t = np.array([[1.5], [1.0], [4.2]])
init_r = np.array([[-1.0, 0.0, 0.0], [0.0, 0.0, -1.0], [0.0, -1.0, 0.0]])
ret, rvec, tvec = cv.solvePnP(obj_pts,
                              img_pts,
                              k,
                              d,
                              useExtrinsicGuess=False,
                              rvec=init_r,
                              tvec=init_t)
ret
R = cv.Rodrigues(rvec)[0]
R_cam1 = R
T_cam1 = tvec

In [ ]:
T_cam1
R_cam1

In [ ]:
# Extrinsic calibration.
ext_path = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/2009_09_11/extrinsic_calib/stereo"
points_fpaths = sorted(glob(os.path.join(ext_path, "points", "points[1-4].json")))
scene_fpath = os.path.join(ext_path, f"{len(points_fpaths)}_cam_scene_test.json")
app.calibrate_arabia_extrinsics(
    scene_fpath, points_fpaths[0], points_fpaths[1],
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/cam1/camera.json",
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/cam2/camera.json",
    R_cam1, T_cam1)


In [ ]:
# Perform SBA on calibration points to refine extrinsic calibration parameters.
data_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/2009_09_11/extrinsic_calib/stereo"
scene_fpath = os.path.join(data_dir, "2_cam_scene_test.json")
points_fpaths = sorted(glob(os.path.join(data_dir, "points", "points[1-4].json")))
scene_sba_fpath = scene_fpath.replace('.json','_sba.json')

res, pts_3d, obj_pts = app.sba_extrinsic_params_standard(
    scene_fpath, points_fpaths, out_fpath=scene_sba_fpath,
    num_view_points=2
)

plt.plot(res['before'], label='Cost Before')
plt.plot(res['after'], label='Cost After')
plt.legend()
plt.show(block=False)

plot_points_3d(pts_3d, "Estimate Before")
plot_points_3d(obj_pts, "Estimate After")

In [ ]:
# Perform SBA on calibration points to refine extrinsic calibration parameters.
data_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_tests/2009_06_18/trail02/extrinsic_calib/manual_calib"
scene_fpath = os.path.join(data_dir, "3_cam_scene_sba.json")
points_fpaths = sorted(glob(os.path.join(data_dir, "points", "points[1-4].json")))
scene_sba_fpath = scene_fpath.replace('.json','_refined.json')

res, pts_3d, obj_pts = app.sba_extrinsic_params_standard(
    scene_fpath, points_fpaths, out_fpath=scene_sba_fpath,
    num_view_points=2
)

plt.plot(res['before'], label='Cost Before')
plt.plot(res['after'], label='Cost After')
plt.legend()
plt.show(block=False)

plot_points_3d(pts_3d, "Estimate Before")
plot_points_3d(obj_pts, "Estimate After")

In [ ]:
# Perform SBA on calibration points to refine extrinsic calibration parameters.
data_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/2009_09_07/extrinsic_calib"
scene_fpath = os.path.join(data_dir, "3_cam_scene.json")
points_fpaths = sorted(glob(os.path.join("/Users/zico/Downloads", "yes", "points[1-4].json")))
scene_sba_fpath = scene_fpath.replace('.json', '_final.json')

res, pts_3d, obj_pts = app.sba_points_and_extrinsic_params_standard(
    scene_fpath,
    points_fpaths,
    out_fpath=scene_sba_fpath,
    robust=True,
    # f_scale=100
)

plt.plot(res['before'], label='Cost Before')
plt.plot(res['after'], label='Cost After')
plt.legend()
plt.show(block=False)

plot_points_3d(pts_3d, "Estimate Before")
plot_points_3d(obj_pts, "Estimate After")

In [ ]:
def plot_scene(scene_fpath, points_dir, ground_plane_pts=None):
    pts_2d, frames = [], []
    points_fpaths = sorted(glob(os.path.join(points_dir, 'points[1-9].json')))
    for fpath in points_fpaths:
        img_pts, img_names, *_ = load_points(fpath)
        pts_2d.append(img_pts)
        frames.append(img_names)
    plot_extrinsics(scene_fpath, pts_2d, frames, triangulate_points, ground_plane_pts=ground_plane_pts, dark_mode=True)


In [ ]:
data_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/2009_09_07/extrinsic_calib/stereo"
plot_scene(os.path.join(data_dir, "2_cam_scene_test.json"), os.path.join("/Users/zico/Downloads", "yes"))
'''
[[0.88411592]
 [0.62751649]
 [4.55060563]]
[[3.84268897]
 [0.70346261]
 [4.42031224]]
'''

In [ ]:
# Moving new points into the existing points.
points = load_json("/Users/zico/Downloads/test2/calb_ext/points3.json")
points_orig = load_json("/Users/zico/Downloads/test2/calb_ext/points/points3_orig.json")
_, frames, board_shape, board_square_len, cam_res = load_points("/Users/zico/Downloads/test2/calb_ext/points/points3_orig.json")
_, new_frames, *_ = load_points("/Users/zico/Downloads/test2/calb_ext/points3.json")
for frame in new_frames:
    if frame in frames:
        points_orig["points"][frame][0][0] = points["points"][frame][0][0]
save_json("/Users/zico/Downloads/test2/calb_ext/points3_.json", points_orig)

In [ ]:
# Quantify calibration with reprojection of points on cheetah.
def grad(p1, p2):
    dx = (p2[0] - p1[0])
    dy = (p2[1] - p1[1])
    dz = (p2[2] - p1[2])
    return dz / dx, dz / dy


data_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/2009_09_11/extrinsic_calib/stereo"
scene_fpath = os.path.join(data_dir, "2_cam_scene_test.json")
data1 = load_json(
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/manual_labels/599/frame1.json"
)
data2 = load_json(
    "/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos/kinetic_dataset/intrinsic_calib/2009_09_11/manual_labels/598/frame1.json"
)
img_pts_1 = []
img_pts_2 = []
for i, point in enumerate(data1["shapes"]):
    if i < 4:
        img_pts_1.append(point["points"][0])
for i, point in enumerate(data2["shapes"]):
    if i < 4:
        img_pts_2.append(point["points"][0])
k_arr, d_arr, r_arr, t_arr, cam_res = load_scene(scene_fpath)
n_cams = len(k_arr)
pts_3d = triangulate_points(
    np.array(img_pts_1), np.array(img_pts_2),
    k_arr[0], d_arr[0], r_arr[0], t_arr[0],
    k_arr[1], d_arr[1], r_arr[1], t_arr[1]
)
# Calculate gradients - Should be close to 0!!
print(f"dz/dy: {grad(pts_3d[0], pts_3d[1])[1]}")  # dy
print(f"dz/dy: {grad(pts_3d[2], pts_3d[3])[1]}") # dy
print(f"dz/dx: {grad(pts_3d[0], pts_3d[3])[0]}") # dx
print(f"dz/dx: {grad(pts_3d[1], pts_3d[2])[0]}") # dx
pts_3d = np.append(pts_3d, [pts_3d[0]]).reshape((-1, 3))
plot_scene(os.path.join(data_dir, "2_cam_scene_test.json"), os.path.join("/Users/zico/Downloads", "yes"), ground_plane_pts=pts_3d)